In [ ]:
# df.loc[idx[slice(None), '2012-02-15'],]['cs_pctrank_ts_argmax5_closeadj'].unique()  -> for indexing multiindex

In [12]:
listy = []
type(listy)

list

In [13]:
import pandas as pd
import nasdaqdatalink as ndl
import json
import numpy as np
from typing import List, Callable, Dict
from realgam.quantlib import general_utils as gu
from realgam.quantlib.engineer.op_engineer_vect import OpEngineerV
from realgam.quantlib.engineer.ta_engineer_vect import TalibEngineerV
from realgam.quantlib.engineer.alpha_engineer_vect import AlphaEngineerV
from realgam.quantlib.engineer.test_no_class import alpha001, ts_ret
import talib
from joblib import Parallel, delayed
import datetime
import os
import time
PROJECT_PATH = os.getenv('QuantSystemMVP')
DATA_PATH = f'{PROJECT_PATH}/Data/historical/stock_hist_perma.obj'
ENG_PATH = f'{PROJECT_PATH}/Data/engineered/e_alpha_20220401.obj'
NDL_CONFIG_PATH = f'{PROJECT_PATH}/realgam/quantlib/nasdaq_dl_config.json'
NDL_CONFIG = json.load(open(NDL_CONFIG_PATH))

In [18]:
data = ndl.get_table('SHARADAR/SEP', ticker='AAPL',
                                     paginate=True, )

In [19]:
data

,ticker,date,open,high,low,close,volume,closeadj,closeunadj,lastupdated
None,,,,,,,,,,
0,AAPL,2022-04-01,174.030,174.880,171.940,174.310,75755805.0,174.310,174.31,2022-04-01
1,AAPL,2022-03-31,177.840,178.030,174.400,174.610,99941081.0,174.610,174.61,2022-03-31
2,AAPL,2022-03-30,178.550,179.610,176.700,177.770,89592832.0,177.770,177.77,2022-03-30
3,AAPL,2022-03-29,176.690,179.010,176.340,178.960,97849001.0,178.960,178.96,2022-03-29
4,AAPL,2022-03-28,172.170,175.730,172.000,175.600,90059845.0,175.600,175.60,2022-03-28
...,...,...,...,...,...,...,...,...,...,...
2575,AAPL,2012-01-09,15.197,15.277,15.048,15.062,394024400.0,12.895,421.73,2022-02-04
2576,AAPL,2012-01-06,14.992,15.098,14.972,15.086,318292800.0,12.916,422.40,2022-02-04
2577,AAPL,2012-01-05,14.820,14.948,14.738,14.930,271269600.0,12.782,418.03,2022-02-04


In [20]:
stocks_df, stocks_extended_df, available_tickers = gu.load_file(DATA_PATH)
stacked_hist = stocks_df.copy()

In [8]:
eng = gu.load_file(ENG_PATH)

In [4]:
tickers = stacked_hist.index.get_level_values('permaticker').unique()[:100]
small_hist = stacked_hist[stacked_hist.index.isin(tickers, level='permaticker')]
# small_hist = stacked_hist[stacked_hist.index.isin(tickers, level='permaticker')]
A = stacked_hist[stacked_hist.index.isin(['A'], level='permaticker')]

In [5]:
oe = OpEngineerV(small_hist)
small_hist['retfwd'] = oe.ts_retfwd(n = 5)

In [10]:
eng = eng.reindex(small_hist.index)

In [22]:
stacked_hist = stacked_hist.reset_index()
stacked_hist.date.max()

Timestamp('2022-04-01 00:00:00')

In [17]:
small_hist.date.max()

Timestamp('2022-03-31 00:00:00')

In [28]:
latest = stacked_hist[stacked_hist.date == pd.to_datetime('2022-04-01')]
list(latest.reindex(latest.volume.nlargest(1000).index).permaticker)

['194273',
 '192721',
 '199991',
 '116141',
 '171049',
 '633589',
 '199059',
 '199562',
 '186639',
 '199713',
 '199495',
 '113479',
 '120599',
 '197676',
 '193211',
 '196754',
 '199626',
 '198738',
 '198215',
 '124571',
 '198808',
 '198739',
 '199475',
 '632369',
 '199941',
 '198314',
 '196785',
 '192549',
 '124373',
 '198508',
 '199960',
 '193578',
 '192364',
 '632043',
 '633922',
 '638392',
 '197620',
 '190594',
 '123532',
 '199144',
 '198445',
 '110473',
 '199650',
 '636166',
 '195256',
 '124302',
 '198399',
 '194817',
 '124419',
 '199428',
 '634849',
 '199739',
 '635864',
 '119395',
 '187429',
 '116017',
 '194897',
 '198616',
 '110842',
 '193197',
 '198061',
 '193333',
 '631756',
 '198214',
 '110813',
 '198461',
 '187914',
 '197750',
 '194976',
 '634353',
 '199853',
 '156969',
 '636670',
 '196257',
 '179780',
 '195656',
 '194474',
 '192873',
 '199345',
 '115953',
 '111025',
 '192490',
 '199252',
 '194823',
 '192603',
 '108442',
 '197145',
 '195691',
 '199839',
 '116537',
 '199891',

In [28]:

# dummy = ae.alpha7().reindex(small_hist.index)
s_time_chunk = time.time()
small_hist['alpha7'] = ae.alpha7().reindex(small_hist.index).values
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 1.3742468357086182 sec


In [29]:
s_time_chunk = time.time()
small_hist['alpha7'] = ae.alpha7()
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 1.309342861175537 sec


In [26]:
small_hist

ticker   open   high    low  close  openadj  highadj  \
permaticker date                                                              
101501      2012-01-03  BBUCQ   0.03   0.03   0.03   0.03     0.03     0.03   
            2012-01-04  BBUCQ   0.03   0.03   0.03   0.03     0.03     0.03   
            2012-01-05  BBUCQ   0.03   0.03   0.03   0.03     0.03     0.03   
            2012-01-06  BBUCQ   0.03   0.03   0.03   0.03     0.03     0.03   
            2012-01-09  BBUCQ   0.03   0.03   0.03   0.03     0.03     0.03   
...                       ...    ...    ...    ...    ...      ...      ...   
110110      2022-03-22   RVMD  21.92  23.90  21.45  23.42    21.92    23.90   
            2022-03-23   RVMD  23.07  24.20  22.51  24.00    23.07    24.20   
            2022-03-24   RVMD  24.03  24.92  23.83  24.69    24.03    24.92   
            2022-03-25   RVMD  25.40  25.96  23.87  24.33    25.40    25.96   
            2022-03-28   RVMD  24.14  24.91  24.14  24.67    24.14    24.91   

                        lowadj  closeadj    volume  alpha7  
permaticker date                                            
101501      2012-01-03    0.03      0.03       0.0     NaN  
            2012-01-04    0.03      0.03       0.0     NaN  
            2012-01-05    0.03      0.03       0.0     NaN  
            2012-01-06    0.03      0.03       0.0     NaN  
            2012-01-09    0.03      0.03       0.0     NaN  
...                        ...       ...       ...     ...  
110110      2022-03-22   21.45     23.42  985655.0   -57.0  
            2022-03-23   22.51     24.00  708893.0   -59.0  
            2022-03-24   23.83     24.69  523315.0    -1.0  
            2022-03-25   23.87     24.33  877947.0   -57.0  
            2022-03-28   24.14     24.67  420701.0    -1.0  

[112505 rows x 11 columns]

In [31]:
PRIMARY_KEY = 'permaticker'
n_std = 20
n_ts_arg_max = 5
df = small_hist.copy()[['closeadj']]
close = df['closeadj'].unstack(PRIMARY_KEY)
oe = OpEngineerV(df)
returns = oe.ts_ret('closeadj', wide=True)

std = oe.ts_std('closeadj', n_std, wide=True)
close[returns < 0] = oe.ts_std('closeadj', n_std, wide=True)
close[std.isnull()] = np.nan
close[returns.isnull()] = np.nan
df['closeadj_square'] = np.power(close, 2).stack().swaplevel()
oe.set_df(df)
oe.ts_argmax('closeadj_square', n_ts_arg_max, inplace=True)
alpha_values = oe.cs_pctrank(f'ts_argmax{n_ts_arg_max}_closeadj_square', wide=True).mul(
-.5).stack().swaplevel()

In [34]:
df['a'] = alpha_values

In [35]:
df

closeadj  closeadj_square    a
permaticker date                                      
199059      2012-01-03    12.574              NaN  NaN
            2012-01-04    12.642              NaN  NaN
            2012-01-05    12.782              NaN  NaN
            2012-01-06    12.916              NaN  NaN
            2012-01-09    12.895              NaN  NaN
...                          ...              ...  ...
            2022-03-21   165.380       27350.5444 -0.5
            2022-03-22   168.820       28500.1924 -0.5
            2022-03-23   170.210       28971.4441 -0.5
            2022-03-24   174.070       30300.3649 -0.5
            2022-03-25   174.720       30527.0784 -0.5

[2575 rows x 3 columns]

In [10]:
df['closeadj'].unstack(PRIMARY_KEY)

permaticker,199059
date,
2012-01-03,12.574
2012-01-04,12.642
2012-01-05,12.782
2012-01-06,12.916
2012-01-09,12.895
...,...
2022-03-21,165.380
2022-03-22,168.820
2022-03-23,170.210


In [4]:
n_adv = 20
PRIMARY_KEY = 'permaticker'
n_delta = 7
n_ts_rank = 60
df = small_hist.copy()[['closeadj', 'volume']]
# df['alpha'] = -1

tae = TalibEngineerV(df)
adv = tae.adv(n_adv, wide=True)
volume = df['volume'].unstack(PRIMARY_KEY)

oe = OpEngineerV(df)

df[f'ts_delta{n_delta}_closeadj'] = oe.ts_delta('closeadj', n_delta)
#
df[f'ts_delta{n_delta}_closeadj_abs'] = df[f'ts_delta{n_delta}_closeadj'].abs()
#
oe.set_df(df)
#
# term_2 = oe.ts_delta(close, 7)
term_2 = df[f'ts_delta{n_delta}_closeadj'].unstack(PRIMARY_KEY)
#
# alpha_values = -1 * oe.ts_rank(f'ts_delta{n_delta}_closeadj_abs', n_ts_rank, wide=True).mul(
#     np.sign(term_2)).where(adv < volume).stack().swaplevel()

In [5]:
-1 * oe.ts_rank(f'ts_delta{n_delta}_closeadj_abs', n_ts_rank, wide=True).mul(
    np.sign(term_2))

permaticker,199059
date,
2012-01-03,NaN
2012-01-04,NaN
2012-01-05,NaN
2012-01-06,NaN
2012-01-09,NaN
...,...
2022-03-21,-31.0
2022-03-22,-57.0
2022-03-23,-59.0


In [85]:
a1 = -oe.ts_rank(f'ts_delta{n_delta}_closeadj_abs', n_ts_rank, wide=True).mul(
    np.sign(term_2))
cond = a1.isnull()
a1[adv >= volume] = -1
a1[cond] = np.nan
a1[adv.isnull() | volume.isnull()] = np.nan


In [86]:
a1

permaticker,199059
date,
2012-01-03,NaN
2012-01-04,NaN
2012-01-05,NaN
2012-01-06,NaN
2012-01-09,NaN
...,...
2022-03-21,-1.0
2022-03-22,-1.0
2022-03-23,-59.0


In [6]:
c0 = 254
c1 = 251
c2 = 256

r_tot = c2/c0 - 1
r_tot2 = (c1/c0) * (c2/c1) - 1

print(f'rtot = {r_tot}, rtot2 = {r_tot2}')

rtot = 0.007874015748031482, rtot2 = 0.007874015748031482


In [50]:
df = small_hist.copy()
n_sum = 10
open = df['openadj'].unstack('ticker')

tae = TalibEngineerV(df)
df['vwap'] = tae.daily_vwap()

oe = OpEngineerV(df)
# term1 = open - (sum(vwap, 10) / 10
df['term1'] = (open - oe.ts_sum('vwap', n_sum, wide=True).divide(n_sum)).stack().swaplevel()
df['close-vwap'] = df['closeadj'] - df['vwap']
oe.set_df(df)
alpha_values = oe.cs_pctrank('term1', wide=True).mul(
    oe.cs_pctrank('close-vwap', wide=True).abs().mul(-1)).stack().swaplevel()

KeyError: 'Level ticker not found'

In [9]:
small_hist['alpha4'] = ae.alpha4()

In [10]:
small_hist

open    high     low  close    openadj    highadj  \
ticker date                                                             
A      2012-01-03  35.84  36.990  35.740  36.48  23.651649  24.410561   
       2012-01-04  36.15  36.360  35.590  36.19  23.855604  23.994184   
       2012-01-05  35.88  37.320  35.720  37.00  23.677891  24.628174   
       2012-01-06  37.09  37.555  36.805  37.40  24.476425  24.783288   
       2012-01-09  37.62  38.420  37.472  38.38  24.826455  25.354397   
...                  ...     ...     ...    ...        ...        ...   
ACER   2022-03-15   2.37   2.410   2.350   2.36   2.370000   2.410000   
       2022-03-16   2.36   2.610   2.360   2.55   2.360000   2.610000   
       2022-03-17   2.55   2.600   2.520   2.57   2.550000   2.600000   
       2022-03-18   2.53   2.670   2.520   2.66   2.530000   2.670000   
       2022-03-21   2.70   2.800   2.640   2.76   2.700000   2.800000   

                      lowadj  closeadj     volume    alpha5  alpha4  
ticker date                                                          
A      2012-01-03  23.585657    24.074  2973100.0       NaN     NaN  
       2012-01-04  23.486056    23.882  3327500.0       NaN     NaN  
       2012-01-05  23.572304    24.417  4894600.0       NaN     NaN  
       2012-01-06  24.288348    24.681  3370100.0       NaN     NaN  
       2012-01-09  24.728786    25.328  3168500.0       NaN     NaN  
...                      ...       ...        ...       ...     ...  
ACER   2022-03-15   2.350000     2.360    27432.0 -0.091681    -4.0  
       2022-03-16   2.360000     2.550   188494.0 -0.171125    -0.0  
       2022-03-17   2.520000     2.570    79341.0 -0.195238    -6.0  
       2022-03-18   2.520000     2.660    74344.0 -0.207927    -8.0  
       2022-03-21   2.640000     2.760   173214.0 -0.403530    -3.0  

[122497 rows x 11 columns]

In [5]:
s_time_chunk = time.time()
close = stacked_hist.closeadj.unstack('ticker')
ret = ts_ret(close)
alpha001(close, ret)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 27.22061824798584 sec


In [4]:
ae = AlphaEngineerV(stacked_hist)

In [7]:
s_time_chunk = time.time()
stacked_hist['alpha1'] = ae.alpha1(5, 20)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 302.45863914489746 sec


In [6]:
s_time_chunk = time.time()
stacked_hist['alpha1'] = ae.alpha1(5, 20)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 319.57942175865173 sec


In [5]:
s_time_chunk = time.time()
stacked_hist['alpha4'] = ae.alpha4(9)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 102.20251607894897 sec
Total time: 6.799350023269653 sec
Total time: 200.1494369506836 sec


In [5]:
s_time_chunk = time.time()
stacked_hist['alpha2'] = ae.alpha2()
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

/Users/punlee/miniconda3/envs/quant/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Total time: 314.89851689338684 sec


In [19]:
volume = stacked_hist.volume.unstack('ticker')

In [6]:
from joblib import Parallel, delayed
from realgam.quantlib.engineer.utils import rank_last_value

In [43]:
def func(series):

    return series.rolling(10).apply(np.argmax, engine='cython', raw=True).add(1)

In [47]:
def rolling_rank(x, window, pct=False, min_prob=None):
    """
    Get the rolling rank of the last value according to the previous window of values.
    """
    # https://github.com/pandas-dev/pandas/issues/9481
    import bottleneck as bk
    norm_rank = bk.move_rank(x, window, axis=0) # [-1, 1]
    u = (norm_rank + 1) / 2 # [0, 1]
    if pct:
        if min_prob is None:
            min_prob = 1 / (window + 1)
            return u * (1 - 2 * min_prob) + min_prob # [min_prob, 1 - min_prob]
    rank = u * (window - 1)
    return np.round(rank)

In [52]:
s_time_chunk = time.time()
dummy = rolling_rank(volume, 10, pct=True)
pd.DataFrame(dummy, index=volume.index, columns=volume.columns)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 0.861375093460083 sec


In [44]:
s_time_chunk = time.time()
values = Parallel(n_jobs=-1)(delayed(func)(volume[col]) for col in volume)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 3.8957040309906006 sec


In [53]:
s_time_chunk = time.time()
volume.rolling(10).apply(rank_last_value, raw=True)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 121.04070401191711 sec


In [33]:
n_total_tickers = volume.shape[1]
batch_size = 20
s_time_chunk = time.time()
values = Parallel(n_jobs=-1)(delayed(func)(volume.iloc[:,i:i + batch_size ]) for i in range(0, n_total_tickers, batch_size))
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 5.859202861785889 sec


In [22]:
from numpy.lib.stride_tricks import sliding_window_view
sw = sliding_window_view(volume, 7, axis=0)
scores_np = (sw <= sw[..., -1:]).sum(axis=2) / sw.shape[-1]
scores_np_df = pd.DataFrame(scores_np, columns=volume.columns)

In [23]:
volume.shape

(2570, 10375)

In [25]:
scores_np

array([[0.14285714, 0.        , 0.        , ..., 0.        , 0.42857143,
        1.        ],
       [0.28571429, 0.        , 0.        , ..., 0.        , 0.71428571,
        1.        ],
       [0.85714286, 0.        , 0.        , ..., 0.        , 0.28571429,
        0.57142857],
       ...,
       [0.28571429, 0.28571429, 0.        , ..., 0.14285714, 0.28571429,
        0.        ],
       [0.14285714, 0.71428571, 0.        , ..., 0.14285714, 1.        ,
        0.        ],
       [1.        , 0.57142857, 0.        , ..., 1.        , 1.        ,
        0.        ]])

In [72]:
pd.concat(values, axis=1)

ticker,A,AA,AAAP,AAC,AAC.U,AACC,AACG,AACH,AACI,AACIU,...,GHRS,GHSEQ,GHSI,GHVIU,GI,GIA,GIA.U,GIAC,GIACU,GIB
date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-14,0.0,2.0,NaN,9.0,9.0,NaN,8.0,NaN,NaN,NaN,...,5.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2022-03-15,0.0,1.0,NaN,7.0,2.0,NaN,2.0,NaN,NaN,NaN,...,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0
2022-03-16,1.0,1.0,NaN,6.0,1.0,NaN,9.0,NaN,NaN,NaN,...,2.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0


In [30]:
open = small_hist.openadj.unstack('ticker')
volume = small_hist.volume.unstack('ticker')
open.rolling(10).corr(volume)

ticker,A,AA,AAAP,AAC,AAC.U,AACC,AACG,AACH,AACI,AACIU,...,IRTC,IRWD,IS,ISAA,ISBC,ISCA,ISCIQ,ISDR,ISEE,ISHCQ
date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-14,0.363984,0.107623,NaN,0.320861,-0.212449,NaN,-0.392003,NaN,NaN,NaN,...,0.113561,-0.518841,0.578930,-0.027211,0.143323,NaN,NaN,0.754544,-0.238503,NaN
2022-03-15,0.485656,0.400572,NaN,0.360201,-0.161166,NaN,-0.355019,NaN,NaN,NaN,...,-0.054785,-0.320664,0.821113,0.010575,-0.120884,NaN,NaN,0.748907,-0.290549,NaN
2022-03-16,0.500228,0.463567,NaN,0.285395,-0.204080,NaN,0.204381,NaN,NaN,NaN,...,-0.286412,0.327993,0.909696,0.076618,0.076101,NaN,NaN,0.730427,-0.141502,NaN


In [18]:
small_hist = small_hist.reset_index().set_index(['ticker', 'date'])

In [9]:
goe = GroupOpEngineer(small_hist, 'ticker')
goe.ts_corr('open','volume',10)

NotImplementedError: isna is not defined for MultiIndex

In [20]:
small_hist.groupby('ticker')['volume'].rolling(10).corr(small_hist['openadj'])

ticker  ticker  date      
A       A       2012-01-03         NaN
                2012-01-04         NaN
                2012-01-05         NaN
                2012-01-06         NaN
                2012-01-09         NaN
                                ...   
ACER    ACER    2022-03-14    0.243021
                2022-03-15    0.388569
                2022-03-16   -0.425023
                2022-03-17   -0.297427
                2022-03-18   -0.259615
Length: 12243500, dtype: float64

In [ ]:
df = small_hist[['closeadj']].reset_index().set_index(['ticker','date'])

In [180]:
goe = GroupOpEngineer(df, 'ticker')

In [181]:
df['ts_ret_closeadj'] = goe.ts_ret('closeadj').values
df['ts_std20_closeadj'] = goe.ts_std('closeadj', 20).values

In [182]:
df.loc[df['ts_ret_closeadj'] < 0, 'closeadj'] = df['ts_std20_closeadj']

In [183]:
df.loc[df['ts_ret_closeadj'].isnull() |  df['ts_std20_closeadj'].isnull(), 'closeadj'] = np.nan

In [184]:
df['closeadj'] = np.power(df.closeadj, 2)

In [185]:
goe.set_df(df)

In [ ]:
asdfas

In [186]:
df['ts_argmax5_closeadj'] = goe.ts_argmax('closeadj', 5).values

In [189]:
goe.set_df(df)
df['cs_pctrank_ts_argmax5_closeadj'] = goe.cs_rank('ts_argmax5_closeadj', method='dense')

In [190]:
df

closeadj  ts_ret_closeadj  ts_std20_closeadj  \
ticker date                                                       
A      2012-01-03       NaN              NaN                NaN   
       2012-01-04       NaN        -0.007975                NaN   
       2012-01-05       NaN         0.022402                NaN   
       2012-01-06       NaN         0.010812                NaN   
       2012-01-09       NaN         0.026214                NaN   
...                     ...              ...                ...   
ACER   2022-03-14  0.000590        -0.024691           0.024288   
       2022-03-15  0.000581        -0.004219           0.024098   
       2022-03-16  6.502500         0.080508           0.030271   
       2022-03-17  6.604900         0.007843           0.030126   
       2022-03-18  7.075600         0.035019           0.030635   

                   ts_argmax5_closeadj  cs_pctrank_ts_argmax5_closeadj  
ticker date                                                             
A      2012-01-03                  NaN                             NaN  
       2012-01-04                  NaN                             NaN  
       2012-01-05                  NaN                             NaN  
       2012-01-06                  NaN                             NaN  
       2012-01-09                  NaN                             NaN  
...                                ...                             ...  
ACER   2022-03-14                  2.0                             3.0  
       2022-03-15                  1.0                             2.0  
       2022-03-16                  5.0                             6.0  
       2022-03-17                  5.0                             6.0  
       2022-03-18                  5.0                             6.0  

[122435 rows x 5 columns]

In [ ]:
print(data.loc[idx[['AAPL','GOOGL'],'2021-03-18'],])

In [168]:
df

closeadj  ts_ret_closeadj  ts_std20_closeadj  \
ticker date                                                       
A      2012-01-03       NaN              NaN                NaN   
       2012-01-04       NaN        -0.007975                NaN   
       2012-01-05       NaN         0.022402                NaN   
       2012-01-06       NaN         0.010812                NaN   
       2012-01-09       NaN         0.026214                NaN   
...                     ...              ...                ...   
ACER   2022-03-14  0.000590        -0.024691           0.024288   
       2022-03-15  0.000581        -0.004219           0.024098   
       2022-03-16  6.502500         0.080508           0.030271   
       2022-03-17  6.604900         0.007843           0.030126   
       2022-03-18  7.075600         0.035019           0.030635   

                   ts_argmax5_closeadj  cs_pctrank_ts_argmax5_closeadj  
ticker date                                                             
A      2012-01-03                  NaN                             NaN  
       2012-01-04                  NaN                             NaN  
       2012-01-05                  NaN                             NaN  
       2012-01-06                  NaN                             NaN  
       2012-01-09                  NaN                             NaN  
...                                ...                             ...  
ACER   2022-03-14                  2.0                            24.5  
       2022-03-15                  1.0                            16.0  
       2022-03-16                  5.0                            43.0  
       2022-03-17                  5.0                            42.5  
       2022-03-18                  5.0                            43.5  

[122435 rows x 5 columns]

In [ ]:
df.loc[idx[slice(None), '2012-02-07'],]['cs_pctrank_ts_argmax5_closeadj']

In [165]:
idx = pd.IndexSlice
df.loc[idx[slice(None), '2012-02-07'],]['cs_pctrank_ts_argmax5_closeadj'].unique()

array([0.44897959, 0.66326531, 0.10204082, 0.85714286, 0.23469388])

In [166]:
df.loc[idx[slice(None), '2012-02-15'],]['cs_pctrank_ts_argmax5_closeadj'].unique()

array([0.10204082, 0.87755102, 0.42857143, 0.67346939, 0.2244898 ])

In [27]:
c = df.closeadj.values
r = df['ts_ret_closeadj'].values
c[r < 0 ] = df['ts_std20_closeadj'].values

ValueError: NumPy boolean array indexing assignment cannot assign 122318 input values to the 55284 output values where the mask is true

In [29]:
c[r < 0]

array([23.882, 25.684, 25.776, ...,  2.43 ,  2.37 ,  2.36 ])